(page:how-to-loading)=
# Loading dat files
As introduced on the previous page, loading of .dat files is handled in xapres using the `from_dats` class in the `xapres.load` module. 

This page will describe in detail a few ways to do this. 

## Unattended and attended ApRES data
ApRES data can be collected in two modes: attended and unattended. 
The most useful way to structure the corresponding datasets is different for each mode.

In attended mode, an ApRES measurement (typically 1 or a few 'bursts', i.e. collections of chirps) are made then the radar is moved or just its antennas are moved, then another measurement if made. Two important ways of distinguishing bursts in an attended survey are the location (which we refer to as a `waypoint`) and the antenna orientation (which we refer to as `orientation`). `waypoint` and `orientation` are therefore both dimensions of the xarray dataset you generate from a set of .dat files collected in attended mode.


In unattended mode, the radar performs repeated measurements at regular intervals autonomously. The timing of the measurements is determined by a configuration file loaded onto the radar by the user. The most important way of distinguishing bursts is therefore the time at which they were collected. `time` is therefore a dimension of the xarray dataset you generate from a set of .dat files collected in unattended mode.

When loading data with XApRES you need to specify which type of survey you are loading. The default is unattended mode. This will be clearer after the example below. 

In [41]:
import sys
sys.path.append("../../../xapres") 
import xapres as xa

## Loading an unattended survey

For both types of data we use the method `.load_all` of the `from_dats` class to collect the data into an xarray dataset.  `.load_all` finds all the .dat files (see previous page), loops through all the chirps in each burst, in each .dat file and loads them into memory. It then computes the profile for each chirp using a fast fourier transform.

For unattended data, `.load_all` concatenates the bursts along the `time` dimension. 

In [42]:
fd_unattended = xa.load.from_dats()
data_unattended = fd_unattended.load_all(directory='../../data/sample/multi-burst-dat-file', bursts_to_process=range(0,7))

100%|██████████| 1/1 [00:00<00:00,  1.43it/s]


```{note}
In the above cell we used the optional parameter `bursts_to_process` to limit the number of bursts loaded. If this is not supplied, all burst will be loaded. You can also provide `file_names_to_process` or `file_numbers_to_process`, to control which .dat files are loaded. 
```

We can see a representation of these data as follows:


In [43]:
data_unattended

<xarray.Dataset> Size: 269MB
Dimensions:          (time: 7, chirp_time: 40001, chirp_num: 20,
                      attenuator_setting_pair: 2, profile_range: 39999)
Coordinates:
  * time             (time) datetime64[ns] 56B 2022-05-22T19:40:20 ... 2022-0...
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 160B 0 1 2 3 4 5 6 ... 14 15 16 17 18 19
    filename         (time) <U62 2kB '../../data/sample/multi-burst-dat-file/...
    burst_number     (time) int64 56B 0 1 2 3 4 5 6
    AFGain           (attenuator_setting_pair) int64 16B -14 -14
    attenuator       (attenuator_setting_pair) float64 16B 10.0 20.0
    orientation      (time) <U7 196B 'unknown' 'unknown' ... 'unknown' 'unknown'
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 90MB ...
    latitude         (time) float64 56B 69.22 69.22 69.22 ... 69.22 69.22 69.22
    longitude        (time) float64 56B -51.12 -51.12 -51.12 ... -51.12 -51.12
    battery_voltage  (time) float64 56B 12.33 12.48 12.51 12.56 12.6 12.56 12.55
    temperature_1    (time) float64 56B 4.383 7.32 7.32 5.75 7.516 8.109 6.734
    temperature_2    (time) float64 56B 5.195 1.469 5.0 3.82 2.844 6.18 5.594
    profile          (time, chirp_num, attenuator_setting_pair, profile_range) complex128 179MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2025-01-06 14:20:27.652025

The dataset has five dimensions:
:`time`: the time of acquisition of each burst of data. In unattended mode data is collected at regular intervals, so it makes sense to have time as a dimension. 
:`chirp_time`: the time since beginning of each chirp of each sample within a chirp. As each chirp lasts 1 second, chirp_time varies from 0 to 1 second. There are 40001 samples by default, so the length of `chirp_time` is 40001. 
:`chirp_num`: this is number of chirps in each burst using each attenuator_setting_pair. So if the ApRES unit was configure to use 2 attenuator setting pairs and make 20 chirps for each in each burst, the total number of chirps in a burst will be 2x20 = 40. 
:`profile_range`: the range to each sample in the profiles computed from each chirp using an fft. 
:`attenuator_setting_pair`: ApRES has two adjustable settings that we refer to here a AFGain and attenuator. For simplicity we refer to them collectively as attenuator settings. For data collection with ApRES in unattended mode, the user can config the system to cycle through multiple pairs of attenuator settings during each burst of data collection. The result is binary files (with the extension .DAT) containing data using different attenuator settings interleaved with one another. Once the data is rearranged in an xarray, these different settings can be selected using the dimension `attenuator_setting_pair`. Note that the dataset we loaded above used just on pair of attenuator settings, so the length of `attenuator_setting_pair` is 1.

The two most important data variables are:
:`chirp`: the 'deramped' chirp. I.e. the signal recorded by the radar after the received signal is mixed with the transmitted signal and low-pass filtered. This is an array of real values. 
:`profile`: the profile resulting from an fast- fourier transform of the chirp. This is an array of complex values.
  
Both of these data variables are 4-D. They share three dimensions: `time`, `chirp_num`, and `attenuator settings_pair`. The variables' fourth dimensions are different. `chirp`'s fourth dimension is the sampling time `chirp_time`. `profile`'s fourth dimension is the range to each measurement `profile_range`.

```{note}
By default, `.load_all` computes profiles and adds them to the dataset To prevent this set `computeProfiles=False`.
```

## Load an attended survey
When operating ApRES in attended mode you manually start individual bursts one after another. Between bursts you move the whole system or change the antenna orientations if you are doing a polarmetric survey. 

As mentioned above, when we load attended data into an xarray dataset it has a different structure than when loading unattended data. The most important difference is that the dimension `time` is replaced by `waypoint`. The time that each burst was recorded is still stored, as it is an important part of the dataset, it just is not a dimension. 

To load ApRES data collected in attended mode you pass the parameter `attended=True` to `.load_all`. You must also provide a list of directories via the `directory` argument corresponding to each waypoint. There should be just one .dat file in each directory. 

Let's load two dat files from two different waypoints on Thwaites Glacier. These data were collected by Elizabeth Case as part of the International Thwaites Glacier Collaboration. 

In [44]:
fd_attended = xa.load.from_dats()
data_attended = fd_attended.load_all(attended=True, 
                     directory=["../../data/sample/thwaites_attended/2022-2023/Point/G1-34-05/", "../../data/sample/thwaites_attended/2022-2023/Point/G1-35-05/"])
data_attended

<xarray.Dataset> Size: 193MB
Dimensions:          (orientation: 1, waypoint: 2, chirp_time: 40001,
                      chirp_num: 100, attenuator_setting_pair: 1,
                      profile_range: 39999)
Coordinates:
    time             (orientation, waypoint) datetime64[ns] 16B 2022-12-31T05...
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (orientation, waypoint) <U91 728B '../../data/sample/thw...
    burst_number     (orientation, waypoint) int64 16B 0 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 26.0
  * orientation      (orientation) <U7 28B 'unknown'
  * waypoint         (waypoint) int64 16B 1 2
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (orientation, waypoint, chirp_time, chirp_num, attenuator_setting_pair) float64 64MB ...
    latitude         (orientation, waypoint) float64 16B 0.0 0.0
    longitude        (orientation, waypoint) float64 16B 0.0 0.0
    battery_voltage  (orientation, waypoint) float64 16B 0.1007 0.06848
    temperature_1    (orientation, waypoint) float64 16B -2.992 -1.82
    temperature_2    (orientation, waypoint) float64 16B -3.516 -4.297
    profile          (orientation, waypoint, chirp_num, attenuator_setting_pair, profile_range) complex128 128MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2025-01-06 14:20:29.253966

In comparison to the unattended data, the two key variables, `chirp` and `profile`, are now functions of `waypoint` rather than `time`. The time of the bursts is stored in the coordinate `time`.

## Load a polarimetric data survey
Another survey design is a polarimetric survey. In this case, the radar is operated in attended mode and the antennas are rotated between bursts. Typically, the user will add HH, HV, VH, or VV to the .dat file names to indicate antenna orientation. To allow `.load_all` to handle this you pass `polarimetric=True` and it will look for  for one .dat file with each of these antenna configurations in each directory and assign the correct orientation to each burst in the xarray dataset. 

In [45]:
fd_polarmetric = xa.load.from_dats()
data_polarimetric = fd_polarmetric.load_all(attended=True, 
                     directory=["../../data/sample/polarmetric"], 
                     polarmetric=True)
data_polarimetric

<xarray.Dataset> Size: 385MB
Dimensions:          (orientation: 4, waypoint: 1, chirp_time: 40001,
                      chirp_num: 100, attenuator_setting_pair: 1,
                      profile_range: 39999)
Coordinates:
    time             (orientation, waypoint) datetime64[ns] 32B 2023-01-05T03...
  * chirp_time       (chirp_time) float64 320kB 0.0 2.5e-05 5e-05 ... 1.0 1.0
  * chirp_num        (chirp_num) int64 800B 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
    filename         (orientation, waypoint) <U56 896B '../../data/sample/pol...
    burst_number     (orientation, waypoint) int64 32B 0 0 0 0
    AFGain           (attenuator_setting_pair) int64 8B -4
    attenuator       (attenuator_setting_pair) float64 8B 22.0
  * orientation      (orientation) <U2 32B 'HH' 'HV' 'VH' 'VV'
  * waypoint         (waypoint) int64 8B 1
  * profile_range    (profile_range) float64 320kB 0.0 0.2103 ... 8.411e+03
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (orientation, waypoint, chirp_time, chirp_num, attenuator_setting_pair) float64 128MB ...
    latitude         (orientation, waypoint) float64 32B 0.0 0.0 0.0 0.0
    longitude        (orientation, waypoint) float64 32B 0.0 0.0 0.0 0.0
    battery_voltage  (orientation, waypoint) float64 32B 0.0 0.0 0.0 0.0
    temperature_1    (orientation, waypoint) float64 32B -9.07 -8.875 ... 2.492
    temperature_2    (orientation, waypoint) float64 32B -10.38 -8.422 ... 4.93
    profile          (orientation, waypoint, chirp_num, attenuator_setting_pair, profile_range) complex128 256MB ...
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2025-01-06 14:20:30.640925

Notice that the `orientation` dimension now has a length of 4. 

## generate_xarray
To simplify using `from_dats.load_all`, we have created a function `generate_xarray` that wraps the instantiation of a `from_dats` object and the call to `load_all`. 


In [46]:
# use from_dats directly
fd_unattended = xa.load.from_dats()
d1 = fd_unattended.load_all(directory='../../data/sample/multi-burst-dat-file', bursts_to_process=range(0,1))

# use generate_xarray
d2 = xa.load.generate_xarray(directory='../../data/sample/multi-burst-dat-file', bursts_to_process=range(0,1))

#confrim they produce the same result
d1.equals(d2)

100%|██████████| 1/1 [00:00<00:00,  9.53it/s]


True

## Summary
`xapres.load.from_dats` handles loading dat files and arranging them in a useful format in xarray datasets. It can handle three survey designs and the sctructure of the dataset that is returnes is slightly different in each case. 

| Survey design | Parameter | Description | most important dimension(s) |
| --- | --- | --- | --- |
| Unattended | attended=False (default) | Data is collected autonomously at regular intervals. | time |
| Attended | attended=True | Data is collected manually at waypoints. | waypoint |
| Polarimetric | polarimetric=True | Data is collected manually at waypoints with different antenna orientations. | waypoint and orientation |
